<a href="https://colab.research.google.com/github/mozzaquatro/financial_market/blob/main/21_AF_Magic%20Formula%20de%20Joel%20Greenblatt%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

2. Obtendo e tratando os dados

In [ ]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [ ]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [ ]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [ ]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
1,CSTB3,150.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
2,PORP4,2.40,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-2,08%",0.0,2.239900e+07,0.00,"13,66%"
3,PMET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"
4,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
980,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
981,GLOB4,30.49,4873.60,8.59,1.512,"0,00%",1.389,-63.21,46.91,-4.59,55.03,28.77,"3,22%","0,03%",0.96,"3,90%","0,18%",0.0,5.675000e+09,1.76,"5,35%"
982,LWSA3,12.92,18050.70,2.58,10.968,"0,21%",1.813,4.51,241.77,7.37,187.94,54.36,"4,54%","0,06%",3.82,"1,33%","0,01%",163323000.0,2.950560e+09,0.04,"53,23%"


tratando os dados

In [ ]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float')/100

In [ ]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,CLAN3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0105,0.0,1.012240e+09,0.00,-0.6396
1,CSTB3,150.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.0,8.420670e+09,0.14,0.3191
2,PORP4,2.40,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0208,0.0,2.239900e+07,0.00,0.1366
3,PMET3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0410,0.0,-2.908630e+08,0.00,0.3774
4,MNSA3,0.42,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,-2.0815,-3.6266,3.63,-0.1350,1.4570,0.0,-9.105000e+06,-6.52,-0.4111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,UBBR11,14.75,1201.81,3.91,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
980,UBBR3,18.00,1466.61,4.77,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
981,GLOB4,30.49,4873.60,8.59,1.512,0.0000,1.389,-63.21,46.91,-4.59,55.03,28.77,0.0322,0.0003,0.96,0.0390,0.0018,0.0,5.675000e+09,1.76,0.0535
982,LWSA3,12.92,18050.70,2.58,10.968,0.0021,1.813,4.51,241.77,7.37,187.94,54.36,0.0454,0.0006,3.82,0.0133,0.0001,163323000.0,2.950560e+09,0.04,0.5323


**Analisando os dados**

empresa com liquidez 
1.000.000 média diária dos ultimos 2 meses

In [ ]:
df_liq2 = df[df['Liq.2meses'] > 1000000]

In [ ]:
df_liq2.shape

(281, 21)

In [ ]:
# 1. P/L (preço da ação dividido pelo lucro por ação).
# não pode ser muito alto (ex.: 10,20,...)
# entra 0 e 10
df_pl = df_liq2[df_liq2['P/L'].between(0,10)]

In [ ]:
df_pl

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
257,CBAV3,12.75,0.00,1.98,0.000,0.0000,0.673,3.86,0.00,-2.56,0.00,0.00,0.0000,0.0000,1.85,0.0000,0.0000,35662200.0,3.843160e+09,0.80,0.0000
260,RECV3,14.66,0.00,1.97,0.000,0.0000,1.100,5.10,0.00,-41.88,0.00,0.00,0.0000,0.0000,2.08,0.0000,0.0000,14551900.0,1.849310e+09,0.43,0.0000
264,DESK3,16.77,0.00,1.68,0.000,0.0000,1.014,2.87,0.00,13.39,0.00,0.00,0.0000,0.0000,4.14,0.0000,0.0000,8356200.0,8.684970e+08,0.45,0.0000
268,BRIT3,5.56,0.00,1.86,0.000,0.0000,0.917,2.71,0.00,54.12,0.00,0.00,0.0000,0.0000,2.81,0.0000,0.0000,5107690.0,1.341070e+09,0.83,0.0000
270,LVTC3,16.50,0.00,1.72,0.000,0.0000,0.692,2.72,0.00,29.21,0.00,0.00,0.0000,0.0000,1.69,0.0000,0.0000,4510150.0,6.161600e+08,0.00,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,TGMA3,15.24,9.36,1.50,0.991,0.0516,0.992,3.65,8.18,7.79,8.03,6.81,0.1211,0.1055,2.39,0.1629,0.1607,7116020.0,6.686340e+08,0.30,-0.0119
653,FESA4,47.07,9.37,1.90,1.915,0.0174,1.330,4.77,6.59,13.48,6.50,5.21,0.2904,0.2044,3.33,0.2462,0.2030,26930400.0,2.185840e+09,0.19,0.1468
656,COCE5,58.00,9.65,1.30,0.617,0.0624,0.416,-1759.15,7.33,-1.06,12.22,7.85,0.0842,0.0639,1.00,0.0681,0.1342,1002950.0,3.486140e+09,0.96,0.1175
657,CARD3,13.36,9.66,1.74,1.110,0.0291,1.001,26.28,6.75,-8.70,6.93,4.02,0.1646,0.1150,1.14,0.1932,0.1807,5205780.0,3.200740e+08,0.35,0.0147


 2. P/VP (preço da ação dividido pelo Valor Patrimonial por ação – informa o quanto o mercado está disposto a pagar sobre o Patrimônio Líquido da empresa).
Quanto maior for o P/VP maior será a bolha especulativa em cima do ativo.
Não pode ser um valor muito alto, não pode ser de 3,4,5,7,20.
Por exemplo, se fosse de 20, seria 20 * o valor do ativo, ou seja, o valor real deveria ser 1 e o mercado está negociando a 20.


In [ ]:
# setando entre 0 e 3
df_pPv = df_pl[df_pl['P/VP'].between(0,5)]

In [ ]:
df_pPv

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
257,CBAV3,12.75,0.00,1.98,0.000,0.0000,0.673,3.86,0.00,-2.56,0.00,0.00,0.0000,0.0000,1.85,0.0000,0.0000,35662200.0,3.843160e+09,0.80,0.0000
260,RECV3,14.66,0.00,1.97,0.000,0.0000,1.100,5.10,0.00,-41.88,0.00,0.00,0.0000,0.0000,2.08,0.0000,0.0000,14551900.0,1.849310e+09,0.43,0.0000
264,DESK3,16.77,0.00,1.68,0.000,0.0000,1.014,2.87,0.00,13.39,0.00,0.00,0.0000,0.0000,4.14,0.0000,0.0000,8356200.0,8.684970e+08,0.45,0.0000
268,BRIT3,5.56,0.00,1.86,0.000,0.0000,0.917,2.71,0.00,54.12,0.00,0.00,0.0000,0.0000,2.81,0.0000,0.0000,5107690.0,1.341070e+09,0.83,0.0000
270,LVTC3,16.50,0.00,1.72,0.000,0.0000,0.692,2.72,0.00,29.21,0.00,0.00,0.0000,0.0000,1.69,0.0000,0.0000,4510150.0,6.161600e+08,0.00,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,TGMA3,15.24,9.36,1.50,0.991,0.0516,0.992,3.65,8.18,7.79,8.03,6.81,0.1211,0.1055,2.39,0.1629,0.1607,7116020.0,6.686340e+08,0.30,-0.0119
653,FESA4,47.07,9.37,1.90,1.915,0.0174,1.330,4.77,6.59,13.48,6.50,5.21,0.2904,0.2044,3.33,0.2462,0.2030,26930400.0,2.185840e+09,0.19,0.1468
656,COCE5,58.00,9.65,1.30,0.617,0.0624,0.416,-1759.15,7.33,-1.06,12.22,7.85,0.0842,0.0639,1.00,0.0681,0.1342,1002950.0,3.486140e+09,0.96,0.1175
657,CARD3,13.36,9.66,1.74,1.110,0.0291,1.001,26.28,6.75,-8.70,6.93,4.02,0.1646,0.1150,1.14,0.1932,0.1807,5205780.0,3.200740e+08,0.35,0.0147


In [ ]:
# 7. ROIC ((Retorno sobre o Capital Investido): calcula-se dividindo o EBIT por 
# (Ativos – Fornecedores – Caixa) – informa ao retorno que a empresa consegue sobre o capital total aplicado).
# deve estar positivo, endividamento da empresa perante o seu investimento
## maior que 0
df_roic = df_pPv[df_pPv['ROIC']>0]
df_roic

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
392,GPIV33,6.63,0.98,0.37,0.574,0.0000,0.177,0.69,0.61,1.53,-0.25,-0.25,0.9368,0.8675,6.79,0.4009,0.3761,2260310.0,1.965900e+09,0.06,0.5519
417,USIM3,13.99,2.13,0.87,0.563,0.0801,0.450,1.24,1.78,3.52,1.66,1.51,0.3157,0.3044,3.08,0.3399,0.4077,5329120.0,2.016430e+10,0.30,0.2709
420,CSNA3,24.13,2.22,1.66,0.707,0.0796,0.440,2.93,1.71,-1.75,2.28,2.00,0.4147,0.3471,1.51,0.3845,0.7469,331582000.0,2.019480e+10,1.47,0.2356
423,USIM5,14.78,2.25,0.92,0.594,0.0834,0.476,1.31,1.88,3.72,1.76,1.60,0.3157,0.3044,3.08,0.3399,0.4077,346769000.0,2.016430e+10,0.30,0.2709
431,GOAU3,10.42,2.77,0.75,0.161,0.2313,0.147,0.47,0.69,3.83,1.13,0.98,0.2318,0.1771,2.80,0.2730,0.2727,2073170.0,1.501220e+10,1.14,0.1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,TGMA3,15.24,9.36,1.50,0.991,0.0516,0.992,3.65,8.18,7.79,8.03,6.81,0.1211,0.1055,2.39,0.1629,0.1607,7116020.0,6.686340e+08,0.30,-0.0119
653,FESA4,47.07,9.37,1.90,1.915,0.0174,1.330,4.77,6.59,13.48,6.50,5.21,0.2904,0.2044,3.33,0.2462,0.2030,26930400.0,2.185840e+09,0.19,0.1468
656,COCE5,58.00,9.65,1.30,0.617,0.0624,0.416,-1759.15,7.33,-1.06,12.22,7.85,0.0842,0.0639,1.00,0.0681,0.1342,1002950.0,3.486140e+09,0.96,0.1175
657,CARD3,13.36,9.66,1.74,1.110,0.0291,1.001,26.28,6.75,-8.70,6.93,4.02,0.1646,0.1150,1.14,0.1932,0.1807,5205780.0,3.200740e+08,0.35,0.0147


In [ ]:
# 8. ROE ((Retorno sobre o Patrimônio Líquido): lucro líquido dividido pelo patrimônio líquido): 
# deve estar positivo, endividamento da empresa perante o seu investimento.
# Não pode ser muito baixo
df_roe = df_roic[df_roic['ROE']>0.1]
df_roe

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
392,GPIV33,6.63,0.98,0.37,0.574,0.0000,0.177,0.69,0.61,1.53,-0.25,-0.25,0.9368,0.8675,6.79,0.4009,0.3761,2260310.0,1.965900e+09,0.06,0.5519
417,USIM3,13.99,2.13,0.87,0.563,0.0801,0.450,1.24,1.78,3.52,1.66,1.51,0.3157,0.3044,3.08,0.3399,0.4077,5329120.0,2.016430e+10,0.30,0.2709
420,CSNA3,24.13,2.22,1.66,0.707,0.0796,0.440,2.93,1.71,-1.75,2.28,2.00,0.4147,0.3471,1.51,0.3845,0.7469,331582000.0,2.019480e+10,1.47,0.2356
423,USIM5,14.78,2.25,0.92,0.594,0.0834,0.476,1.31,1.88,3.72,1.76,1.60,0.3157,0.3044,3.08,0.3399,0.4077,346769000.0,2.016430e+10,0.30,0.2709
431,GOAU3,10.42,2.77,0.75,0.161,0.2313,0.147,0.47,0.69,3.83,1.13,0.98,0.2318,0.1771,2.80,0.2730,0.2727,2073170.0,1.501220e+10,1.14,0.1310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,TGMA3,15.24,9.36,1.50,0.991,0.0516,0.992,3.65,8.18,7.79,8.03,6.81,0.1211,0.1055,2.39,0.1629,0.1607,7116020.0,6.686340e+08,0.30,-0.0119
653,FESA4,47.07,9.37,1.90,1.915,0.0174,1.330,4.77,6.59,13.48,6.50,5.21,0.2904,0.2044,3.33,0.2462,0.2030,26930400.0,2.185840e+09,0.19,0.1468
656,COCE5,58.00,9.65,1.30,0.617,0.0624,0.416,-1759.15,7.33,-1.06,12.22,7.85,0.0842,0.0639,1.00,0.0681,0.1342,1002950.0,3.486140e+09,0.96,0.1175
657,CARD3,13.36,9.66,1.74,1.110,0.0291,1.001,26.28,6.75,-8.70,6.93,4.02,0.1646,0.1150,1.14,0.1932,0.1807,5205780.0,3.200740e+08,0.35,0.0147


In [ ]:
# dívida liquida / patrimonio
# dívida liquida deve ser menor que 20% do patrimonio liquido
df_d20p = df_roe[df_roe['Dív.Brut/ Patrim.']< 0.3]
df_d20p

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
392,GPIV33,6.63,0.98,0.37,0.574,0.0000,0.177,0.69,0.61,1.53,-0.25,-0.25,0.9368,0.8675,6.79,0.4009,0.3761,2260310.0,1.965900e+09,0.06,0.5519
447,ETER3,17.98,3.29,2.05,1.045,0.0000,1.137,2.13,3.76,4.09,3.01,2.85,0.2782,0.3172,3.81,0.4399,0.6232,21549800.0,5.409700e+08,0.07,0.0939
457,CESP6,23.10,3.62,1.01,3.511,0.1124,0.520,26.20,6.39,-1.26,7.68,5.71,0.5497,0.9693,1.37,0.0849,0.2790,67981600.0,7.485270e+09,0.26,0.1003
484,ALLD3,15.46,4.24,0.95,0.251,0.0295,0.347,1.51,3.93,3.78,3.77,3.19,0.0640,0.0593,1.47,0.1942,0.2232,6555170.0,1.523640e+09,0.27,0.0000
498,ENAT3,13.73,4.61,1.05,2.796,0.0141,0.630,1.78,9.97,6.59,5.86,2.10,0.2803,0.6067,3.47,0.1135,0.2284,21002700.0,3.467240e+09,0.26,0.3068
512,LAVV3,5.10,5.14,0.78,1.564,0.1374,0.546,1.03,5.11,1.28,2.23,2.23,0.3057,0.3475,4.42,0.1624,0.1517,8518520.0,1.362660e+09,0.04,0.0000
563,LPSB3,2.46,6.53,1.65,1.636,0.0000,0.827,3.66,6.81,-8.27,4.38,3.28,0.2402,0.3343,2.25,0.1777,0.2532,3501560.0,2.194150e+08,0.00,0.1822
578,CRPG5,69.04,6.94,2.30,2.055,0.0714,1.677,3.36,8.86,4.84,7.91,7.15,0.2318,0.2959,5.15,0.2433,0.3315,3318770.0,8.702190e+08,0.00,0.1256
582,ITSA4,9.97,7.14,1.34,10.733,0.0474,1.035,29.57,62.44,-12.01,65.61,42.71,0.1719,1.6203,1.59,0.0177,0.1876,316571000.0,6.260200e+10,0.13,0.1013
583,ITSA3,9.99,7.15,1.34,10.754,0.0473,1.037,29.63,62.56,-12.04,65.73,42.79,0.1719,1.6203,1.59,0.0177,0.1876,2465630.0,6.260200e+10,0.13,0.1013


menor relacao EV/Ebit e
maior ROI

In [ ]:
ranking = pd.DataFrame()
ranking['pos']=range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [ ]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,CMIN3
1,2,GOAU3,WIZS3
2,3,GOAU4,PSSA3
3,4,USIM3,TASA4
4,5,EVEN3,TASA3
...,...,...,...
145,146,VULC3,SEER3
146,147,VIIA3,DIRR3
147,148,ALSO3,EZTC3
148,149,LREN3,COCE5


In [ ]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [ ]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [ ]:
t = pd.concat([a,b])
t

,AALR3,ABEV3,ALLD3,ALSO3,ALUP11,AMAR3,ASAI3,AURA33,B3SA3,BEEF3,BLAU3,BMOB3,BRFS3,BRKM5,BRML3,CAML3,CARD3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,COCE5,CPFE3,CPLE11,CPLE3,CPLE6,CRFB3,CRPG5,CSED3,CSMG3,CSNA3,CURY3,CYRE3,DEXP3,DIRR3,DMMO3,DXCO3,ECOR3,...,TASA3,TASA4,TECN3,TEND3,TGMA3,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP3,UNIP6,USIM3,USIM5,VALE3,VBBR3,VIIA3,VITT3,VIVT3,VLID3,VULC3,WIZS3,AERI3,ALPA4,AMBP3,ARZZ3,BOAS3,INTB3,MOSI3,MULT3,ORVR3,PETZ3,RADL3,RDOR3,RRRP3,TFCO4,VAMO3,VIVA3,WEGE3
pos,144.0,139,26,147.0,21,96,116,76.0,102,51,121,87,115,10,137.0,86,81,112,80,15,75,59,11,106,66,48,45,49,88,94,97,54,9,47,98,61,55,22,91,73,...,28,27,33,67,84,82,141,72,38,74,133,37,36,5,6,8,100,146.0,126,136.0,107,140,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,NaN,62,46,NaN,40,118,57,NaN,37,36,14,48,127,15,NaN,84,47,110,133,126,108,107,1,143,86,124,123,125,98,28,136,101,11,34,137,55,141,12,69,79,...,5,4,83,121,59,120,49,117,63,91,139,9,8,17,16,6,80,NaN,43,NaN,148,129,2,114.0,39.0,144.0,96.0,95.0,56.0,38.0,149.0,150.0,105.0,99.0,111.0,112.0,25.0,82.0,53.0,30.0


In [ ]:
rank = t.dropna(axis=1).sum()
rank

ABEV3     201
ALLD3      72
ALUP11     61
AMAR3     214
ASAI3     173
         ... 
VBBR3     180
VITT3     169
VLID3     255
VULC3     269
WIZS3      18
Length: 133, dtype: int64

In [ ]:
rank.sort_values()[:15]

PSSA3     4
CMIN3    12
VALE3    14
WIZS3    18
CSNA3    20
ETER3    21
USIM5    22
USIM3    22
BRKM5    25
GOAU3    26
GOAU4    26
MRFG3    29
TASA4    31
TASA3    33
GGBR3    34
dtype: int64